# 🤖 Basic OpenAI API Usage

By the end of this notebook, you'll be able to:

1. **Content Moderation** - Check user messages for inappropriate content
2. **Text Generation** - Draft responses, summarize documents, assist with troubleshooting
3. **Audio Features** - Convert text to speech and transcribe audio
4. **Image Generation** - Create simple visuals for documentation
5. **Vision Analysis** - Analyze error screenshots and hardware images
6. **Cost Management** - Use the most cost-effective models for each task

---

## 🌐 What is an API?

An **API (Application Programming Interface)** is a way for different software programs to communicate with each other.

### 🎯 What is an API Endpoint?

An **endpoint** is a specific function or service the API provides. For example:
- `/v1/chat/completions` - Text generation endpoint
- `/v1/moderations` - Content moderation endpoint
- `/v1/images/generations` - Image generation endpoint

Each endpoint has a specific purpose and accepts different parameters.

### 📬 Request/Response Structure

API communication follows a simple pattern:

1. **Request**: You send data to the API
   - Headers (authentication, content type)
   - Parameters (instructions, settings)
   - Body (the actual data to process)

2. **Response**: The API sends back results
   - Status code (200 = success, 429 = rate limit, etc.)
   - Data (the result you requested)
   - Metadata (usage stats, IDs, etc.)

### ⚠️ Important Concepts

**Rate Limits**: APIs limit how many requests you can make per minute/day to prevent abuse. If you exceed these limits, you'll receive an error.

**Costs**: Most API calls cost money based on usage:
- Text models charge per "token" (roughly 4 characters)
- Image models charge per image generated
- Audio models charge per character (TTS) or per minute (transcription)

**Security Best Practice**: ⚠️ **NEVER hardcode API keys in production code!** Always use environment variables, secrets management, or secure configuration systems.

---

Let's get started!

---

# 🔧 Setup

First, we need to configure your OpenAI API key and install required libraries.

## 📦 Install Dependencies

We'll install three libraries:
- **openai**: Official OpenAI Python client for API access
- **pillow**: Image processing library for displaying and manipulating images
- **requests**: HTTP library for downloading files from URLs

In [ ]:
!pip install -q openai pillow requests

## 🔑 API Key Configuration

You have two methods to provide your API key:

**Method 1 (Recommended)**: Use Colab Secrets
1. Click the 🔑 icon in the left sidebar
2. Click "Add new secret"
3. Name: `OPENAI_API_KEY`
4. Value: Your OpenAI API key
5. Enable notebook access

**Method 2 (Fallback)**: Manual input when prompted

Run the cell below to configure authentication:

In [ ]:
import os

# Configure OpenAI API key
# Method 1: Try to get API key from Colab secrets (recommended)
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    # Method 2: Manual input (fallback)
    from getpass import getpass
    print("💡 To use Colab secrets: Go to 🔑 (left sidebar) → Add new secret → Name: OPENAI_API_KEY")
    OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Validate that the API key is set
if not OPENAI_API_KEY or OPENAI_API_KEY.strip() == "":
    raise ValueError("❌ ERROR: No API key provided!")

print("✅ Authentication configured!")

# Configure which OpenAI model to use
# Options: "gpt-4o", "gpt-4o-mini", "gpt-4-turbo", "gpt-3.5-turbo", "gpt-5-nano", etc.
OPENAI_MODEL = "gpt-5-nano"  # Using gpt-5-nano for cost efficiency
print(f"🤖 Selected Model: {OPENAI_MODEL}")

## 🚀 Initialize OpenAI Client

Now let's create a client instance to interact with the OpenAI API:

In [ ]:
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ OpenAI client initialized successfully!")

---

# 1️⃣ Content Moderation

## 📖 What it Does

The moderation API detects potentially harmful or inappropriate content in text. It checks for:
- Hate speech
- Harassment and bullying
- Violence and graphic content
- Self-harm content
- Sexual content
- And more...

## 🎯 When to Use It

Use moderation **before** processing user-submitted content:
- Support ticket messages
- Chat messages
- Email content
- User feedback forms

## 💡 Why It Matters

1. **Protects your API account** - OpenAI can suspend accounts that process harmful content
2. **Ensures safe interactions** - Filters out inappropriate content before your team sees it
3. **FREE to use** - The moderation API is completely free for OpenAI users!

## 🔑 Key Parameters

- **input**: The text to check (string or array of strings)

**Response contains:**
- **flagged**: Boolean indicating if content violates policies
- **categories**: Dictionary of specific violation types (hate, violence, etc.)
- **category_scores**: Confidence scores (0-1) for each category

---

## 💻 Practical Example: Checking Customer Messages

In [ ]:
# Example 1: Frustrated but appropriate customer message
customer_message_1 = """This is absolutely ridiculous! Your service is terrible
and I'm furious about this issue with my account!"""

# Check the message using moderation API
response = client.moderations.create(input=customer_message_1)

result = response.results[0]

print("🔍 Moderation Check Results:")
print(f"Is Flagged: {result.flagged}")
print(f"\nCategory Flags:")
for category, flagged in result.categories:
    if flagged:
        print(f"  ⚠️ {category}: {flagged}")

if not result.flagged:
    print("\n✅ Message is safe to process")

In [ ]:
# Example 2: Inappropriate message (DO NOT USE IN PRODUCTION)
# This example demonstrates what gets flagged
inappropriate_message = """I hate your company and all your stupid employees.
You're all idiots and deserve to lose your jobs."""

response = client.moderations.create(input=inappropriate_message)
result = response.results[0]

print("🔍 Moderation Check Results:")
print(f"Is Flagged: {result.flagged}")
print(f"\nCategory Flags:")
for category, flagged in result.categories:
    if flagged:
        print(f"  ⚠️ {category}: {flagged}")

if result.flagged:
    print("\n🚫 Message contains inappropriate content - DO NOT process with API")

## 🛠️ Reusable Function

Let's create a simple function you can use in your projects:

In [ ]:
def check_content_safety(text):
    """
    Check if text content is safe to process.

    Args:
        text (str): The text to check

    Returns:
        dict: Contains 'is_safe' boolean and 'flagged_categories' list
    """
    response = client.moderations.create(input=text)
    result = response.results[0]

    # Get all flagged categories
    flagged_categories = [category for category, flagged in result.categories if flagged]

    return {
        "is_safe": not result.flagged,
        "flagged_categories": flagged_categories
    }

# Test the function
test_message = "My printer won't work and I need help urgently!"
safety_check = check_content_safety(test_message)

print(f"Is safe: {safety_check['is_safe']}")
if safety_check['flagged_categories']:
    print(f"Flagged for: {', '.join(safety_check['flagged_categories'])}")
else:
    print("No violations detected ✅")

## ⚠️ Common Pitfalls

1. **Not checking all category flags individually** - Just checking `flagged` isn't enough; sometimes you need to know *what* was flagged

2. **Forgetting to check moderation before sending to main API** - Always moderate user content first to protect your account

3. **Over-relying on moderation** - The API is very good but not 100% perfect. Consider it as a helpful filter, not absolute protection

4. **Not handling edge cases** - Very short messages or special characters might behave unexpectedly

---

---

# 2️⃣ Text Generation

## 📖 What it Does

Text generation models create human-like text responses using advanced language models. They can:
- Answer questions
- Draft responses
- Summarize content
- Provide troubleshooting guidance
- Explain technical concepts

## 🎯 When to Use It

- **Drafting support ticket responses** - Save time with AI-generated first drafts
- **Summarizing documentation** - Condense long technical docs into key points
- **Troubleshooting assistance** - Generate step-by-step diagnostic procedures
- **Knowledge base creation** - Turn technical info into user-friendly content

## 💰 Cost Implications

Text generation is charged **per token**.

**What's a token?** Roughly 4 characters or 0.75 words. For example:
- "Hello" = 1 token
- "Hello, how are you?" = 5 tokens
- 100 words ≈ 133 tokens

You're charged for **both input and output tokens**.

## 📏 Context Window

Models have limits on how much text they can process at once (input + output). For example:
- `gpt-5-nano`: 128K tokens (~96,000 words)
- `gpt-3.5-turbo`: 16K tokens (~12,000 words)

## 🔑 Key Parameters for gpt-5-nano (Responses API)

**Important**: gpt-5-nano uses the **Responses API**, not the Chat Completions API!

- **model**: "gpt-5-nano" (one of the cheapest models)
- **input**: Your prompt or question (string)
- **text**: Dictionary with optional settings:
  - **verbosity**: "low", "medium", or "high" - controls response length and detail
    - "low": Brief, concise responses
    - "medium": Balanced responses (default)
    - "high": Detailed, comprehensive responses

**Response structure**:
- Use `response.output_text` to get the generated text
- Use `response.usage.total_tokens` for token count

**Example**:
```python
response = client.responses.create(
    model="gpt-5-nano",
    input="Your prompt here",
    text={"verbosity": "high"}
)

output = response.output_text
```



---

## 💻 Example 1: Drafting a Support Ticket Response

In [ ]:
# Scenario: User reports printer not working
user_ticket = """My printer won't print anything. I tried turning it off and on
but nothing works. I need to print reports for a meeting in 30 minutes."""

# System prompt defines the assistant's role and behavior
system_prompt = """You are an IT support assistant. Respond professionally,
empathetically, and provide clear troubleshooting steps. Keep responses concise but thorough."""

# Combine system prompt with user message
full_input = f"{system_prompt}\n\nUser message: {user_ticket}"

# Make the API call using Responses API
response = client.responses.create(
    model=OPENAI_MODEL,
    input=full_input,
    text={"verbosity": "high"}
)

# Extract the response
draft_response = response.output_text

print("📝 Draft Response:")
print(draft_response)
print(f"\n📊 Tokens used: {response.usage.total_tokens}")

## 💻 Example 2: Summarizing Technical Documentation

In [ ]:
# Scenario: Summarize a long technical document
long_technical_doc = """
VPN Configuration Guide:

To establish a secure VPN connection, users must first ensure they have the latest
VPN client installed (version 8.2 or higher). The installation package can be
downloaded from the company portal under Software > Security Tools.

Once installed, launch the VPN client and enter your corporate credentials.
The username should be in the format: firstname.lastname@company.com.
Use your standard Windows password.

After successful authentication, select the appropriate VPN gateway from the dropdown:
- US-EAST-01: For users in North America
- EU-WEST-01: For users in Europe
- ASIA-PAC-01: For users in Asia Pacific

Click Connect and wait for the status indicator to turn green. This typically takes
15-30 seconds. Once connected, you'll have access to internal resources including
file shares, internal websites, and database servers.

If you experience connection issues, first verify your internet connection is stable.
Then check if your antivirus software is blocking the VPN client. Common antivirus
programs that may interfere include McAfee and Norton. Add an exception for the
VPN client executable if necessary.

For persistent issues, contact IT support with your error message and the VPN.log
file located in C:\\Program Files\\CompanyVPN\\logs\\.
"""

system_prompt = "Summarize this technical documentation in 3-4 bullet points for end users."

# Combine system prompt with document
full_input = f"{system_prompt}\n\n{long_technical_doc}"

response = client.responses.create(
    model=OPENAI_MODEL,
    input=full_input,
    text={"verbosity": "medium"}  # Using medium for concise summary
)

summary = response.output_text

print("📋 Summary:")
print(summary)
print(f"\n📊 Tokens used: {response.usage.total_tokens}")
print(f"   Input: {response.usage.input_tokens}, Output: {response.usage.output_tokens}")

## 💻 Example 3: Troubleshooting Assistant

In [ ]:
# Scenario: User describes an error
user_description = "I keep getting 'Access Denied' when trying to open the shared drive."

system_prompt = """You are a troubleshooting assistant. Provide step-by-step diagnostic steps.
Number each step clearly. Focus on the most common causes first."""

# Combine system prompt with user description
full_input = f"{system_prompt}\n\nUser issue: {user_description}"

response = client.responses.create(
    model=OPENAI_MODEL,
    input=full_input,
    text={"verbosity": "high"}
)

troubleshooting_steps = response.output_text

print("🔧 Troubleshooting Steps:")
print(troubleshooting_steps)

## 🛡️ Error Handling

Always wrap API calls in try/except blocks to handle potential errors:

In [ ]:
def generate_response_safely(user_message, system_message="You are a helpful IT assistant."):
    """
    Generate a response with proper error handling.

    Args:
        user_message (str): The user's input
        system_message (str): System prompt for the assistant

    Returns:
        dict: Contains 'success' boolean, 'response' text, and optional 'error' message
    """
    try:
        # Combine system prompt with user message
        full_input = f"{system_message}\n\nUser: {user_message}"

        response = client.responses.create(
            model=OPENAI_MODEL,
            input=full_input,
            text={"verbosity": "high"}
        )

        return {
            "success": True,
            "response": response.output_text,
            "tokens_used": response.usage.total_tokens
        }

    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "response": None
        }

# Test the function
result = generate_response_safely("How do I reset my password?")

if result["success"]:
    print("✅ Response generated successfully")
    print(result["response"])
else:
    print(f"❌ Error: {result['error']}")

## ⚠️ Common Pitfalls

1. **Vague prompts leading to generic responses**
   - ❌ Bad: "Help with printer"
   - ✅ Good: "Provide step-by-step troubleshooting for a printer that won't print"

2. **Not setting appropriate max_completion_tokens**
   - Set limits to control costs and prevent overly long responses
   - 100 tokens ≈ 75 words

3. **Forgetting to handle API errors**
   - Always use try/except blocks
   - Common errors: rate limits, invalid API key, network issues

4. **Not using system prompts effectively**
   - System prompts set the behavior and tone
   - Be specific about desired output format and style

5. **Using unsupported parameters**
   - `gpt-5-nano` only supports the default temperature (1)
   - Don't specify temperature parameter or you'll get an error

---

---

# 3️⃣ Audio Features (Text-to-Speech & Transcription)

## 📖 What it Does

OpenAI provides two audio capabilities:
1. **Text-to-Speech (TTS)**: Convert text into natural-sounding speech
2. **Transcription**: Convert audio recordings into text

## 🎯 When to Use It

**Text-to-Speech:**
- Creating audio guides for common procedures
- Accessibility features for visually impaired users
- Automated phone system messages
- Training materials

**Transcription:**
- Converting support call recordings to text
- Creating searchable records of meetings
- Documenting verbal troubleshooting sessions

## 💰 Models

**TTS Models:**
- `tts-1`: Faster, cheaper, good quality
- `tts-1-hd`: Higher quality, more expensive

**Transcription Model:**
- `whisper-1`: OpenAI's speech recognition model

---

## 🔊 Part A: Text-to-Speech

### 🔑 Key Parameters

- **model**: `"tts-1"` (cheaper, faster) or `"tts-1-hd"` (higher quality)
- **voice**: Choose from 6 voices:
  - `alloy`: Neutral, balanced
  - `echo`: Male, clear
  - `fable`: Male, expressive
  - `onyx`: Male, deep
  - `nova`: Female, energetic
  - `shimmer`: Female, soft
- **input**: Text to convert (max 4096 characters)
- **speed**: 0.25 to 4.0 (default 1.0)

### 💻 Practical Example: Create Password Reset Audio Guide

In [ ]:
# Text for our audio guide
password_reset_guide = """
Welcome to the password reset guide. Here are the steps:

First, go to the login page and click 'Forgot Password'.

Second, enter your work email address.

Third, check your email for a reset link. This may take a few minutes.

Finally, click the link and create a new password.

Remember: your password must be at least 8 characters with numbers and symbols.

If you need further assistance, contact the IT help desk. Thank you.
"""

# Generate speech
print("🎙️ Generating audio...")
response = client.audio.speech.create(
    model="tts-1",  # Using cheaper model
    voice="nova",   # Female, energetic voice
    input=password_reset_guide,
    speed=1.0
)

# Save the audio file
audio_file_path = "/content/password_reset_guide.mp3"
response.stream_to_file(audio_file_path)

print(f"✅ Audio saved to: {audio_file_path}")
print("\n▶️ You can play it using the file browser on the left sidebar")

In [ ]:
# Display audio player in notebook
from IPython.display import Audio, display

display(Audio(audio_file_path))

## 🎤 Part B: Transcription (Speech-to-Text)

### 🔑 Key Parameters

- **model**: `"whisper-1"` (OpenAI's speech recognition model)
- **file**: Audio file to transcribe (max 25MB)
- **language**: Optional ISO-639-1 code (e.g., "en" for English) for better accuracy
- **response_format**: "json", "text", "srt", "vtt", or "verbose_json"

**Supported formats**: mp3, mp4, mpeg, mpga, m4a, wav, webm

### 💻 Practical Example: Transcribe the Audio We Just Created

In [ ]:
# Transcribe the password reset guide we just created
print("🎤 Transcribing audio...")

with open(audio_file_path, "rb") as audio_file:
    transcript = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        language="en"  # Specify English for better accuracy
    )

print("✅ Transcription complete!\n")
print("📝 Transcribed Text:")
print(transcript.text)

### 🔄 Compare Original vs Transcription

In [ ]:
print("=" * 60)
print("COMPARISON: Original Text → Audio → Transcription")
print("=" * 60)

print("\n📄 ORIGINAL TEXT:")
print(password_reset_guide)

print("\n🎙️ TRANSCRIBED TEXT:")
print(transcript.text)

print("\n✨ Notice how accurate the transcription is!")

## ⚠️ Common Pitfalls

### Text-to-Speech:
1. **Text too long** - Maximum 4096 characters per request. Split longer text into chunks.
2. **Wrong speed settings** - Speed too fast (>1.5) can reduce clarity
3. **Not choosing appropriate voice** - Test different voices for your use case

### Transcription:
1. **Audio files too large** - Whisper-1 has a 25MB limit. Compress large files first.
2. **Wrong audio format** - Ensure your file is in a supported format (mp3, wav, etc.)
3. **Not specifying language** - Adding the language parameter improves accuracy
4. **Poor audio quality** - Background noise and low volume reduce transcription accuracy
5. **Very long files** - Consider splitting files longer than 30 minutes

---

---

# 4️⃣ Image Generation

## 📖 What it Does

The image generation API creates images from text descriptions using DALL-E models. You provide a text prompt, and it generates an original image.

## 🎯 When to Use It

- **Quick mockups** - Visualize concepts before creating proper designs
- **Documentation visuals** - Create diagrams and illustrations for guides
- **Training materials** - Generate images for presentations and tutorials
- **Placeholder images** - Quick visuals during development


## 💰 Models

- `dall-e-3`: Latest DALL-E model, high quality, better prompt following (recommended)
- `dall-e-2`: Cheaper, good quality, faster

**We'll use DALL-E 3 for better quality and prompt understanding.**

## 🔑 Key Parameters

- **model**: `"dall-e-3"` (recommended) or `"dall-e-2"`
- **prompt**: Description of the image (max 4000 characters for DALL-E 3)
- **size**: Image dimensions
  - DALL-E 3: `"1024x1024"` (square), `"1792x1024"` (landscape), or `"1024x1792"` (portrait)
  - DALL-E 2: `"256x256"`, `"512x512"`, or `"1024x1024"`
- **quality**: `"standard"` or `"hd"` (DALL-E 3 only, default: "standard")
  - "hd": Creates images with finer details and greater consistency (costs more)
  - "standard": Regular quality, more cost-effective
- **style**: `"vivid"` or `"natural"` (DALL-E 3 only)
  - "vivid": Hyper-real, dramatic images
  - "natural": More natural-looking images
- **n**: Number of images to generate
  - DALL-E 3: **Only supports n=1** (one image per request)
  - DALL-E 2: Supports 1-10 images per request
- **response_format**: `"url"` (default) or `"b64_json"`


---

## 💻 Practical Example: Network Diagram for Documentation

In [ ]:
# Create a simple network diagram
prompt = """A simple network diagram showing a router connected to a firewall,
with three workstations behind it. Clean, technical style with clear labels.
Minimalist and professional. White background."""

print("🎨 Generating image...")
print(f"Prompt: {prompt}\n")

response = client.images.generate(
    model="dall-e-3",
    prompt=prompt,
    size="1024x1024",      # Square format for documentation
    quality="standard",    # Standard quality for cost efficiency
    style="natural",       # Natural style for technical diagrams
    n=1
)

# Get the image URL
image_url = response.data[0].url
print(f"✅ Image generated!")
print(f"URL: {image_url}")

# DALL-E 3 provides a revised/enhanced prompt
print(f"\n📝 Revised prompt: {response.data[0].revised_prompt}")

In [ ]:
# Download and display the image
import requests
from PIL import Image
from io import BytesIO

# Download the image
image_response = requests.get(image_url)
image = Image.open(BytesIO(image_response.content))

# Save to file
image_path = "/content/network_diagram.png"
image.save(image_path)
print(f"💾 Saved to: {image_path}")

# Display in notebook
display(image)

---

# 5️⃣ Vision (Image Analysis)

## 📖 What it Does

Vision-capable models can "see" and analyze images. They can:
- Describe what's in an image
- Read text from screenshots
- Identify error messages
- Analyze hardware components
- Recognize UI elements


## 💰 Models

- `gpt-5-nano`: Cost-efficient vision model (recommended for most use cases)
- `gpt-4o`: Highest quality vision analysis


**Image input options:**
- URL: `{"type": "input_image", "image_url": "https://..."}`
- Base64: `{"type": "input_image", "image_url": "data:image/jpeg;base64,..."}`

**Key differences from text-only API:**
- Use `input` parameter (list format) instead of simple string
- Content is an array with `input_text` and `input_image` objects
- Access output with `response.output_text` (same as text API)

---

## 💻 Practical Example: Error Screenshot Analysis

### 📸 Upload Your Image

To use this example:
1. Click the 📁 folder icon in the left sidebar
2. Click the upload button (📤)
3. Upload an error screenshot or any IT-related image
4. Update the `image_path` variable below with your filename

In [ ]:
# Method 1: Analyze an uploaded image file
import base64

# Update this path with your uploaded image
image_path = "/content/error_screenshot.png"  # Change this to your image filename

# Check if file exists
import os
if not os.path.exists(image_path):
    print("⚠️ Image file not found!")
    print("Please upload an image file and update the image_path variable above.")
    print("\nFor demonstration, we'll create a simple example...")
else:
    # Read and encode the image
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    # Analyze the image using Responses API
    response = client.responses.create(
        model="gpt-5-nano",
        input=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "input_text",
                        "text": """Please analyze this screenshot:
                        1. What error or issue is shown?
                        2. What are the likely causes?
                        3. What troubleshooting steps would you recommend?"""
                    },
                    {
                        "type": "input_image",
                        "image_url": f"data:image/png;base64,{base64_image}"
                    }
                ]
            }
        ],
        text={"verbosity": "high"}
    )

    analysis = response.output_text

    print("🔍 Image Analysis:")
    print("=" * 60)
    print(analysis)
    print("=" * 60)
    print(f"\n📊 Tokens used: {response.usage.total_tokens}")

## ⚠️ Common Pitfalls

1. **Image files too large**
   - Recommended: Keep images under 20MB
   - Compress large images before uploading
   - Use tools like PIL to resize: `image.thumbnail((1024, 1024))`

2. **Unclear or low-resolution screenshots**
   - Ensure text in screenshots is readable
   - Higher resolution = better analysis
   - Avoid blurry or pixelated images

3. **Not providing enough context in the text prompt**
   - ❌ Bad: "What's in this image?"
   - ✅ Good: "What error is shown in this Windows screenshot and what are possible solutions?"

4. **Assuming the model can see very small text**
   - If text is important, make sure it's legible in the screenshot
   - Zoom in or crop to the relevant area

5. **Incorrect image encoding**
   - Ensure proper base64 encoding
   - Include correct MIME type (image/png, image/jpeg, etc.)

6. **Not handling file path errors**
   - Always check if file exists before trying to read it
   - Use try/except blocks for error handling

---

## 💡 Cost Optimization Tips

In [ ]:
# Example: Calculate approximate cost for a text generation request

def estimate_text_cost(input_text, output_tokens_estimate=200, model="gpt-5-nano"):
    """
    Estimate the cost of a text generation request.

    Args:
        input_text (str): The input text
        output_tokens_estimate (int): Expected output length in tokens
        model (str): Model name

    Returns:
        dict: Cost breakdown
    """
    # Rough token estimation (actual tokenization is more complex)
    input_tokens = len(input_text) // 4

    # Pricing (per 1M tokens)
    if model == "gpt-5-nano":
        input_cost_per_1m = 0.05
        output_cost_per_1m = 0.40
    elif model == "gpt-4o":
        input_cost_per_1m = 2.50
        output_cost_per_1m = 10.00
    else:
        input_cost_per_1m = 0.05  # Default to nano pricing
        output_cost_per_1m = 0.40

    input_cost = (input_tokens / 1_000_000) * input_cost_per_1m
    output_cost = (output_tokens_estimate / 1_000_000) * output_cost_per_1m
    total_cost = input_cost + output_cost

    return {
        "input_tokens": input_tokens,
        "output_tokens_estimate": output_tokens_estimate,
        "total_tokens": input_tokens + output_tokens_estimate,
        "input_cost": input_cost,
        "output_cost": output_cost,
        "total_cost": total_cost
    }

# Example
sample_ticket = "My computer won't start. The screen is black and I hear beeping sounds."
cost = estimate_text_cost(sample_ticket, output_tokens_estimate=300)

print("💰 Cost Estimate:")
print(f"Input tokens: {cost['input_tokens']}")
print(f"Output tokens (estimated): {cost['output_tokens_estimate']}")
print(f"Total tokens: {cost['total_tokens']}")
print(f"\nEstimated cost: ${cost['total_cost']:.6f}")
print(f"  Input: ${cost['input_cost']:.6f}")
print(f"  Output: ${cost['output_cost']:.6f}")

## ✅ Best Practices Summary

### 1. Content Safety
- ✅ **Always use moderation for user-generated content**
- ✅ Check moderation BEFORE sending to other APIs
- ✅ It's FREE - use it liberally!

### 2. Model Selection
- ✅ **Choose the cheapest model that meets your needs**
  - Text: `gpt-5-nano` for most tasks (uses Responses API)
  - Audio TTS: `tts-1` (not `tts-1-hd`)
  - Images: `dall-e-3` for high quality, `dall-e-2` for cost savings
  - Vision: `gpt-4o` (currently the main vision model)
- ✅ Only upgrade to premium models when quality difference matters

### 3. API Usage for gpt-5-nano
- ✅ **Use the Responses API**: `client.responses.create()`
- ✅ **Access output correctly**: `response.output_text`
- ✅ **Control verbosity**: Use `text={"verbosity": "low|medium|high"}`
  - "low": Brief responses, saves tokens
  - "medium": Balanced (default)
  - "high": Detailed, comprehensive responses

### 4. Image Generation with DALL-E 3
- ✅ **Remember n=1 limitation**: DALL-E 3 only generates 1 image per request
- ✅ **Use quality wisely**: "standard" for most cases, "hd" only when needed
- ✅ **Choose appropriate style**: "vivid" for graphics, "natural" for realistic images
- ✅ **Check revised_prompt**: See how DALL-E 3 enhanced your prompt

### 5. Error Handling
- ✅ **Always use try/except blocks**
- ✅ Handle rate limits gracefully
- ✅ Provide helpful error messages to users
- ✅ Log errors for debugging

### 6. Security
- ✅ **NEVER hardcode API keys in production**
- ✅ Use environment variables or secrets management
- ✅ Rotate API keys periodically
- ✅ Set up usage alerts in OpenAI dashboard

### 7. Monitoring
- ✅ **Track your API usage regularly**
- ✅ Set up billing alerts
- ✅ Monitor which endpoints cost the most
- ✅ Review and optimize high-usage areas

### 8. Prompt Engineering
- ✅ Be specific about what you want
- ✅ Include system instructions directly in the input
- ✅ Provide examples when helpful
- ✅ Iterate and improve prompts based on results

### 9. Model-Specific Settings
- ✅ `gpt-5-nano` uses the **Responses API**:
  ```python
  response = client.responses.create(
      model="gpt-5-nano",
      input="your prompt here",
      text={"verbosity": "high"}
  )
  output = response.output_text
  ```
- ✅ Much cheaper than gpt-4o
- ✅ DALL-E 3 provides automatic prompt enhancement via `revised_prompt`

---



# 7️⃣ Mini-Project: Support Ticket Analyzer

## 🎯 Project Description

Let's combine everything we've learned! We'll build a **Support Ticket Analyzer** that:

1. ✅ **Checks if the ticket content is appropriate** (Moderation)
2. 📊 **Analyzes the urgency and issue type** (Text Generation)
3. ✉️ **Drafts a professional response** (Text Generation)

This is a practical tool you could adapt for real IT support workflows!

---

## 🛠️ TODO Version (Try It Yourself First!)

Try to complete the function below on your own. Fill in the TODO sections using what you've learned.

In [ ]:
def analyze_support_ticket_TODO(ticket_content):
    """
    Analyze a support ticket and generate a response.

    Args:
        ticket_content (str): The customer's ticket message

    Returns:
        dict: Analysis results including safety check, urgency, issue type, and draft response
    """

    # TODO 1: Check content safety with moderation API
    # Hint: Use client.moderations.create()
    # Store result in a variable called 'is_safe'

    # If content is not safe, return early
    # if not is_safe:
    #     return {"is_safe": False, "message": "Inappropriate content detected"}

    # TODO 2: Analyze ticket urgency and issue type
    # Create a prompt that asks the AI to identify:
    #   - Urgency level (Low, Medium, High, Critical)
    #   - Issue type (Hardware, Software, Network, Access, Other)
    # Hint: Use client.chat.completions.create() with a good system prompt

    # TODO 3: Generate a professional response
    # Create another AI call to draft a response to the customer
    # The response should be:
    #   - Professional and empathetic
    #   - Address the specific issue mentioned
    #   - Provide clear next steps

    return {
        "is_safe": True,  # Replace with actual result
        "urgency": "TODO",
        "issue_type": "TODO",
        "draft_response": "TODO"
    }

print("🎓 TODO: Complete the function above, then test it with the test cases below!")

## ✅ Complete Solution

Here's a full implementation of the support ticket analyzer:

In [ ]:
def analyze_support_ticket(ticket_content):
    """
    Analyze a support ticket and generate a response.

    Args:
        ticket_content (str): The customer's ticket message

    Returns:
        dict: Analysis results including safety check, urgency, issue type, and draft response
    """

    try:
        # Step 1: Check content safety
        print("🔍 Checking content safety...")
        moderation_response = client.moderations.create(input=ticket_content)
        is_safe = not moderation_response.results[0].flagged

        if not is_safe:
            flagged_categories = [
                category for category, flagged
                in moderation_response.results[0].categories
                if flagged
            ]
            return {
                "is_safe": False,
                "flagged_categories": flagged_categories,
                "message": "⚠️ Inappropriate content detected. Ticket requires manual review."
            }

        print("✅ Content is safe\n")

        # Step 2: Analyze urgency and issue type
        print("📊 Analyzing urgency and issue type...")
        analysis_prompt = f"""You are an IT ticket classification assistant. Be concise.

Analyze this support ticket and provide:
1. Urgency level: Critical / High / Medium / Low
2. Issue type: Hardware / Software / Network / Access / Account / Other

Provide your response in this exact format:
Urgency: [level]
Issue Type: [type]

Ticket: {ticket_content}
"""

        analysis_response = client.responses.create(
            model=OPENAI_MODEL,
            input=analysis_prompt,
            text={"verbosity": "low"}  # Low verbosity for classification
        )

        analysis_text = analysis_response.output_text

        # Parse the response
        urgency = "Medium"  # Default
        issue_type = "Other"  # Default

        for line in analysis_text.split('\n'):
            if 'urgency:' in line.lower():
                urgency = line.split(':')[1].strip()
            elif 'issue type:' in line.lower():
                issue_type = line.split(':')[1].strip()

        print(f"  Urgency: {urgency}")
        print(f"  Issue Type: {issue_type}\n")

        # Step 3: Generate professional response
        print("✍️ Drafting response...")
        response_prompt = f"""You are a helpful IT support professional.

Draft a professional IT support response to this ticket.

Guidelines:
- Be empathetic and professional
- Acknowledge the issue
- Provide clear troubleshooting steps or next actions
- Include estimated response time if urgent
- Keep it concise (3-4 short paragraphs)

Ticket urgency: {urgency}
Issue type: {issue_type}

Ticket content: {ticket_content}
"""

        response_generation = client.responses.create(
            model=OPENAI_MODEL,
            input=response_prompt,
            text={"verbosity": "high"}
        )

        draft_response = response_generation.output_text

        print("✅ Analysis complete!\n")

        return {
            "is_safe": True,
            "urgency": urgency,
            "issue_type": issue_type,
            "draft_response": draft_response,
            "success": True
        }

    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "message": f"❌ Error analyzing ticket: {str(e)}"
        }

print("✅ Support Ticket Analyzer function created!")

## 🧪 Test Cases

Let's test the analyzer with different types of tickets:

### Test 1: Urgent Password Reset Request

In [ ]:
test_ticket_1 = """
I've been locked out of my account and can't reset my password!
The password reset link isn't working and I have an important
presentation in 1 hour. I really need access ASAP!
"""

print("=" * 70)
print("TEST 1: Urgent Password Reset")
print("=" * 70)
print(f"\n📝 Ticket Content:\n{test_ticket_1}\n")

result = analyze_support_ticket(test_ticket_1)

if result['success']:
    print("=" * 70)
    print("📊 ANALYSIS RESULTS")
    print("=" * 70)
    print(f"Safe: {result['is_safe']}")
    print(f"Urgency: {result['urgency']}")
    print(f"Issue Type: {result['issue_type']}")
    print(f"\n✉️ DRAFT RESPONSE:")
    print("-" * 70)
    print(result['draft_response'])
    print("-" * 70)
else:
    print(result['message'])

### Test 2: General Software Question

In [ ]:
test_ticket_2 = """
Hi, I'm trying to understand how to use the shared calendar feature
in Outlook. Can someone explain how to share my calendar with my team
and set the appropriate permissions? Thanks!
"""

print("=" * 70)
print("TEST 2: General Software Question")
print("=" * 70)
print(f"\n📝 Ticket Content:\n{test_ticket_2}\n")

result = analyze_support_ticket(test_ticket_2)

if result['success']:
    print("=" * 70)
    print("📊 ANALYSIS RESULTS")
    print("=" * 70)
    print(f"Safe: {result['is_safe']}")
    print(f"Urgency: {result['urgency']}")
    print(f"Issue Type: {result['issue_type']}")
    print(f"\n✉️ DRAFT RESPONSE:")
    print("-" * 70)
    print(result['draft_response'])
    print("-" * 70)
else:
    print(result['message'])

### Test 3: Frustrated User (Professional but Upset)

In [ ]:
test_ticket_3 = """
This is the third time this month the VPN has gone down! I'm trying to
work from home and I can't access any company resources. This is extremely
frustrating and affecting my productivity. When will this be fixed properly?
"""

print("=" * 70)
print("TEST 3: Frustrated User")
print("=" * 70)
print(f"\n📝 Ticket Content:\n{test_ticket_3}\n")

result = analyze_support_ticket(test_ticket_3)

if result['success']:
    print("=" * 70)
    print("📊 ANALYSIS RESULTS")
    print("=" * 70)
    print(f"Safe: {result['is_safe']}")
    print(f"Urgency: {result['urgency']}")
    print(f"Issue Type: {result['issue_type']}")
    print(f"\n✉️ DRAFT RESPONSE:")
    print("-" * 70)
    print(result['draft_response'])
    print("-" * 70)
else:
    print(result['message'])

---

# 🚨 Error Handling Reference

Here are common errors you might encounter and how to handle them:

---

## Common Error Types

In [ ]:
from openai import OpenAIError, RateLimitError, APIError, AuthenticationError

def make_robust_api_call(prompt, max_retries=3):
    """
    Make an API call with comprehensive error handling.

    Args:
        prompt (str): The prompt to send
        max_retries (int): Maximum number of retry attempts

    Returns:
        dict: Response or error information
    """
    import time

    for attempt in range(max_retries):
        try:
            response = client.responses.create(
                model=OPENAI_MODEL,
                input=prompt,
                text={"verbosity": "high"}
            )

            return {
                "success": True,
                "response": response.output_text
            }

        # Error 1: Rate Limit - Too many requests
        except RateLimitError as e:
            wait_time = 2 ** attempt  # Exponential backoff: 1s, 2s, 4s...
            print(f"⚠️ Rate limit hit. Waiting {wait_time} seconds...")
            if attempt < max_retries - 1:
                time.sleep(wait_time)
                continue
            return {
                "success": False,
                "error_type": "rate_limit",
                "message": "Rate limit exceeded. Please try again later."
            }

        # Error 2: Authentication - Invalid API key
        except AuthenticationError as e:
            return {
                "success": False,
                "error_type": "authentication",
                "message": "Invalid API key. Please check your credentials."
            }

        # Error 3: API Error - OpenAI service issues
        except APIError as e:
            print(f"⚠️ API error occurred: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2)
                continue
            return {
                "success": False,
                "error_type": "api_error",
                "message": f"OpenAI API error: {str(e)}"
            }

        # Error 4: Any other errors
        except Exception as e:
            return {
                "success": False,
                "error_type": "unknown",
                "message": f"Unexpected error: {str(e)}"
            }

    return {
        "success": False,
        "error_type": "max_retries",
        "message": "Maximum retry attempts exceeded."
    }

print("✅ Robust API call function created!")
print("\nThis function handles:")
print("  1. Rate limits with exponential backoff")
print("  2. Authentication errors")
print("  3. API service errors")
print("  4. Unknown errors")
print("  5. Automatic retries")

## Test the Error Handling

In [ ]:
# Test with a normal request
result = make_robust_api_call("What is an IP address?")

if result['success']:
    print("✅ Success!")
    print(f"Response: {result['response']}")
else:
    print(f"❌ Error ({result['error_type']}): {result['message']}")

## 📋 Error Reference Table

| Error Type | Common Cause | Solution |
|------------|--------------|----------|
| **Rate Limit** | Too many requests per minute | Implement exponential backoff, reduce request frequency |
| **Authentication** | Invalid/expired API key | Check API key, regenerate if needed |
| **Model Not Found** | Typo in model name | Verify model name matches OpenAI docs |
| **Token Limit Exceeded** | Input/output too long | Reduce prompt length or max_completion_tokens |
| **Timeout** | Request took too long | Increase timeout, check network connection |
| **Invalid Request** | Missing required parameters | Check API documentation for required fields |
| **Content Policy** | Violated OpenAI policies | Use moderation API first, adjust content |

---



## 📖 Resources

- **OpenAI Cookbook**: https://cookbook.openai.com
- **API Reference**: https://platform.openai.com/docs/api-reference
- **Best Practices**: https://platform.openai.com/docs/guides/production-best-practices
- **Pricing**: https://openai.com/pricing


